In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from prompt_info import PromptInfo
from import_seg import import_seg_prompt
from semantic_seg import semantic_seg_prompt
from task_seg import task_seg_prompt
import torch
import json
print(task_seg_prompt.answer_1)


In [2]:
class PromptInfo1:
    def __init__(self, system="你是一个优秀的人工智能" , que_1="问题示例1", answer_1="答案1", que_2="问题示例2 ", answer_2="答案2"):
        # 大模型的角色定位描述
        self.system = system
        # 示例 1 的输入
        self.que_1 = que_1
        # 示例 1 的输出
        self.answer_1 = answer_1
        # 示例 2 的输入
        self.que_2 = que_2
        # 示例 2 的输出
        self.answer_2 = answer_2

In [ ]:
pro=PromptInfo1("nishish")
print(pro.system)

In [ ]:
device = "cuda" # the device to load the model onto

#model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/LLM-Research/Meta-Llama-3-8B-Instruct"
#model_path = "/home/imc/桌面/aas_rag_generate/Qwen/Qwen2-7B"
model_path = "/home/imc/桌面/Qwen1.5-72B/pythonProject/model/qwen/Qwen1___5-14B-Chat"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [5]:
def generate_response(text):
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    input_ids = tokenizer.encode(text,return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape,dtype=torch.long,device=device)
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
# Prepare your prompts
prompt = "scara机械臂将水瓶夹起放置在传送带上，经传送带运输到delata机器人前"
print(semantic_seg_prompt.system)
messages = [
    {"role": "system", "content": semantic_seg_prompt.system},
    {"role": "user", "content": semantic_seg_prompt.que_1},
    {"role": "assistant", "content": semantic_seg_prompt.answer_1},
    {"role": "user", "content": semantic_seg_prompt.que_2},
    {"role": "assistant", "content": semantic_seg_prompt.answer_2},
    {"role": "user", "content": prompt}

]
semantic_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(semantic_seg_text)

# response = generate_response(text=semantic_seg_text)
# #print(response)

# data = json.loads(response)

# print(data)

with open("./semantic_result_demo.json", 'r') as file:
    equipment_required_data = json.load(file)

print(equipment_required_data)

In [7]:
def get_matched_asset_name(equipment_name,aas_data):
    #to do
    return equipment_name

def get_digital_twin_submodle_byname(asset_name,aas_json):
    for asset_item in aas_json:
        if(asset_item["assetName"]==asset_name):
            return asset_item    
    return {}  

In [ ]:
# 加载aas模型json文件
with open("./aas_model.json", 'r') as file:
    aas_data = json.load(file)
    print(aas_data)
equipment_info = equipment_required_data["equipmentInfo"]
#print(equipment_info)

#根据equipment name匹配asset，增加json字段，后续要多次使用
for item in equipment_info:
    equipment_name = item["equipmentName"]
    asset_name = get_matched_asset_name(equipment_name,aas_data)
    item["aas_sub_model"] = get_digital_twin_submodle_byname(asset_name, aas_data)
    item["asset_name"] = asset_name 
    for istance in item["instanceEquipment"]:
        istance["aas_sub_model"] = item["aas_sub_model"]
        istance["asset_name"] = asset_name
        print(istance)
    print(item)

print(get_digital_twin_submodle_byname("jetbot 小车",aas_data))
print("jetbot小车")
# item["aas_model"]='''    {
#         "assetName":"jetbot 小车",
#         "assetDescription":"一个用于移动的智能小车",
#         "libRelyOn":["lib3"],
#         "initSeg":"init_jetbot",
#         "resetSeg":"reset_jetbot",
#         "physicsSeg":"physics_jetbot"
#     }
# '''

# 遍历每一个equipment中的equipmentName字段

# 此部分代码根据json文本的描述，生成相应的init段等的代码

# 查询 根据每种equipment的描述词，查询出aas模型资料库中对应的字段信息


# 查询到字段，输出相应的import、 init、 reset等段提示词信息

In [ ]:
##import段的操作
# Prepare your prompts
import_seg_user_input=" "
for item in equipment_info:
    print(item)
    asset_name = item["asset_name"]
    import_seg_user_input += item["aas_sub_model"]["importSeg"]

print(import_seg_user_input)



messages = [
    {"role": "system", "content": import_seg_prompt.system},
    {"role": "user", "content": import_seg_prompt.que_1},
    {"role": "assistant", "content": import_seg_prompt.answer_1},
    {"role": "user", "content": import_seg_prompt.que_2},
    {"role": "assistant", "content": import_seg_prompt.answer_2},
    {"role": "user", "content": import_seg_user_input}

]
import_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(import_seg_text)

import_seg_output = generate_response(import_seg_text)
print(import_seg_output)


In [ ]:
# init seg

##定位
init_seg_system = "你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段"

init_seg_text = []

## 模板
init_seg_user_input_template = "请你写出将{object}放在{location}并初始化的代码，变量名请使用{variable_name}或以{variable_name}作为前缀"

# 使用 format 方法替换占位符
object_name = "方块"
location = "桌子上"
variable_name = "block"
prefix = "block_"


formatted_input = init_seg_user_input_template.format(object=object_name, location=location, variable_name=variable_name)
print(formatted_input)

for item in equipment_info :
    for instance in item["instanceEquipment"] :
        init_seg_user_input = init_seg_user_input_template.format(object=instance["asset_name"], location=instance["position"], variable_name=instance["programName"])
        print(init_seg_user_input)
        messages = [
            {"role": "system", "content": init_seg_system},
            {"role": "user", "content": instance["aas_sub_model"]["initSeg"]["initDescription"]},
            {"role": "assistant", "content": instance["aas_sub_model"]["initSeg"]["initCode"]},
            {"role": "user", "content": init_seg_user_input}
        ]
        init_seg_text_temp = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        init_seg_text.append(init_seg_text_temp)


print(init_seg_text)

init_seg_output = []
# for text in init_seg_text :
#     init_seg_output.append(generate_response(text))

In [ ]:
# reset seg
# reset seg

##定位
reset_seg_system = "你是一名高级的python工程师，能够根据需求和经验写出正确的代码片段"

reset_seg_text = []

## 模板
reset_seg_user_input_template = "请你写出{object}的reset代码，变量名请使用{variable_name}或以{variable_name}作为前缀"



for item in equipment_info :
    for instance in item["instanceEquipment"] :
        reset_seg_user_input = reset_seg_user_input_template.format(object=instance["asset_name"], variable_name=instance["programName"])
        print(reset_seg_user_input)
        messages = [
            {"role": "system", "content": reset_seg_system},
            {"role": "user", "content": instance["aas_sub_model"]["resetSeg"]["resetDescription"]},
            {"role": "assistant", "content": instance["aas_sub_model"]["resetSeg"]["resetCode"]},
            {"role": "user", "content": reset_seg_user_input}
        ]
        reset_seg_text_temp = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        reset_seg_text.append(reset_seg_text_temp)


print(reset_seg_text)

reset_seg_output = []
# for text in reset_seg_text :
#     reset_seg_output.append(generate_response(text))

In [12]:
def get_matched_instance_by_description(description, equipment_info_json) :
    for item in equipment_info_json :
        for instance in item["instanceEquipment"] :
            if instance["descriptionName"]==description :
                return instance
            
    return {}
    

In [ ]:
#任务切分
# Prepare your prompts
task_seg_user_input="franka机械臂抓起一个物块A放到jetbot小车上，ur10机械臂将物块A抓起放到另一个位置 " #后边看看要不要处理一下吧

# print(task_seg_user_input)

# print(task_seg_prompt.answer_1)

messages = [
    {"role": "system", "content": task_seg_prompt.system},
    {"role": "user", "content": task_seg_prompt.que_1},
    {"role": "assistant", "content": task_seg_prompt.answer_1},
    {"role": "user", "content": task_seg_prompt.que_2},
    {"role": "assistant", "content": task_seg_prompt.answer_2},
    {"role": "user", "content": task_seg_user_input}
]
task_seg_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# print(task_seg_text)

task_seg_output = generate_response(task_seg_text)
print(task_seg_output)

task_seg_json = json.loads(task_seg_output)
print(task_seg_json["task_count"])



In [14]:
def get_matched_physcis_seg_by_task_description(task_description, asset_used) :
    # to do 
    return asset_used[0]["instance"]["aas_sub_model"]["physicsSeg"]


In [15]:
def generate_phy_seg_que_template(task_description,variable_list):
    template = "请你写出{}的Python代码片段，其中{}"
    variable_names = ", ".join([f"{var_name}使用变量名{var}" for var_name, var in variable_list])
    init_seg_user_input = template.format(task_description, variable_names)
    return init_seg_user_input

In [ ]:
task_set = task_seg_json["task_set"]
for task in task_set :
    for asset in task["asset_used"] :
        asset["instance"] = get_matched_instance_by_description(asset["user_description"],equipment_info)

print(task_seg_json)

phy_seg_prompt_system = "你是一个高级pyhton工程师，能够学习历史对话中的示例，写出符合用户输入需求的python程序片段"
phy_seg_text = []
# 模板
phy_seg_user_input_template = "请你写出{object}的python代码片段，其中变量名请使用{variable_name}或以{variable_name}作为前缀"

for task in task_set :
    task_number = task["task_number"]
    phy_seg_reference = get_matched_physcis_seg_by_task_description(task["task_description"],task["asset_used"])
    print(phy_seg_reference)
    phy_template_variables = []
    for asset in task["asset_used"] :
        phy_template_variables.append((asset["user_description"],asset["instance"]["programName"]))

    phy_seg_user_input = generate_phy_seg_que_template(task["task_description"],phy_template_variables)
    messages = [
    {"role": "system", "content": phy_seg_prompt_system},
    {"role": "user", "content": phy_seg_reference["physicsDescription"]},
    {"role": "assistant", "content": phy_seg_reference["physicsCode"]},
    {"role": "user", "content": phy_seg_user_input},
    ]
    phy_seg_text_temp = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )
    phy_seg_text.append(phy_seg_text_temp)

print(phy_seg_text[1])
phy_seg_output = []

# for text in phy_seg_text :
#     phy_seg_output.append(generate_response(text))


    


In [17]:
# 将代码拼接起来